# 🐦 Twitter Sentiment Analysis in Spanish Tweets  

## 👥 Authors
- **Kristopher Javier Alvarado López** — Carné: 21188  
- **Emilio Jose Solano Orozco** — Carné: 21212  

---

## 📖 Project Description
Este proyecto consiste en el desarrollo de un **pipeline completo de Procesamiento de Lenguaje Natural (NLP)** aplicado a tweets en español.  
El objetivo es analizar el sentimiento expresado en los mensajes (positivo, negativo o neutro) integrando las siguientes etapas:  

1. **Preprocesamiento del texto**: limpieza, tokenización, lematización, normalización y corrección ortográfica.  
2. **Representación de texto**: Bag of Words (BoW), TF-IDF, matrices de co-ocurrencia y embeddings.  
3. **Modelos probabilísticos**: construcción de modelos N-gram y evaluación con entropía y perplejidad.  
4. **Modelos de clasificación**: aplicación de algoritmos supervisados para la detección automática de sentimiento.  
5. **Evaluación y discusión crítica** de resultados.  

---

## 🌍 English Abstract
This project presents the development of a **complete NLP pipeline** for sentiment analysis in Spanish tweets.  
The main goal is to evaluate different text preprocessing techniques, representation models, and machine learning approaches to classify tweets into sentiment categories (positive, negative, neutral). The project combines classical probabilistic language models with modern embedding-based techniques, offering a critical analysis of their performance on real-world social media data.


In [ ]:
import pandas as pd

# Cargar dataset
df = pd.read_csv("data/sentiment_analysis_dataset.csv")

# Vista rápida
print("Columnas originales:", df.columns.tolist())
print("Tamaño del dataset:", df.shape)
df.head()
